In [2]:
import sys
import random
import pickle

def get_player_move(player, board):
    try:
        move = input("Player " + str(player) + "'s turn: ")
        move = move.strip().split(',')
        move = [int(x) for x in move]
        return move
    except:
        print("error reading input")
        return False
        
    
def check_move_valid(player, move, board):
    if len(move) != 2:
        print("Cannot read move format!")
        return False
    
    for pos in move:
        if pos > 2:
            print("You moved outside the edges of the board!")
            return False
        
    board_pos = 3 * move[1] + move[0]
    if board[board_pos] != None:
        print("That spot was already taken!")
        return False
    
    return True

def add_move_to_board(player, move, board):
    board_pos = 3 * move[1] + move[0]
    board[board_pos] = player
    
# return value = [bool gameover, bool computerwin]
def check_game_over(board):
    computer_win = False
    game_over = False
    
    # check for x axis wins, so like x-x-x all in a row
    for x in range(0,3):
        if board[3*x] != None and board[3*x] == board[3*x+1] and board[x] == board[3*x+2]:
            game_over = True
            if board[x] == 2:
                computer_win = True
        
    # check for y axis wins, like above but vertically oriented
    for y in range(0,3):
        if board[y] != None and board[y] == board[y+3] and board[y] == board[y+6]:
            game_over = True
            if board[y] == 2:
                computer_win = True
    
    # check for upwards diagonal wins, 0,2-1,1-2,0
    if board[6] == board[4] and board[6] == board[2]:
        game_over = True
        if board[4] == 2:
            computer_win = True
    
    # check for downwards diagonal wins, 0,0-1,1-2,2
    if board[0] == board[4] and board[0] == board[8]:
        game_over = True
        if board[4] == 2:
            computer_win = True
            
    return [game_over, computer_win]


def switch_player(player):
    if player == 1:
        return 2
    else:
        return 1
    
# returns the move so we can add it to the list of moves for this game
def get_computer_move(board, past_boards):
    print("Computer's turn...")
    
#     board_pos = random_move(board)
    board_pos = move_from_past_moves(board, past_boards)
    
    move = [board_pos % 3, int(board_pos / 3)]
    add_move_to_board(2, move, board)
    return move

def move_from_past_moves(board, past_boards):
    # choose from the highest scored positions in past_boards
    current_board_key = past_boards_key(board)
    if current_board_key in past_boards:
        scores = past_boards[current_board_key]
        for score in sorted(scores, reverse=True):
            for x in range(0, len(board)):
                if scores[x] == score and board[x] == None:
                    return x
    else:
        return random_move(board)


def past_boards_key(board):
    boards_key = []
    for pos in board:
        if pos == None:
            boards_key.append('-')
        else:
            boards_key.append(str(pos))
    return ''.join(boards_key)
    
def random_move(board):
    available = []
    for x in range(0, len(board)):
        if board[x] == None:
            available.append(x)
            
    return random.choice(available)

def print_board(board):
    for y in range(0, 3):
        for x in range(0, 3):
            if board[3*y+x] == 1:
                sys.stdout.write(' X ')
            elif board[3*y+x] == 2:
                sys.stdout.write(' O ')
            else:
                sys.stdout.write('   ')
            
            if x != 2:
                sys.stdout.write('|')
        if y != 2:
            print("\n---+---+---")
        else:
            sys.stdout.write("\n")
    

def score_past_boards(past_boards, this_games_input, game_over_result):
    # take each move in the game input list
    # create a board from the cumulative moves
    # when the current move is a computer move
        # get the key for this board
        # if there is a board in past boards for this key,
            # score the move position based on what the outcome of the game was  
    this_game_board = [None] * 9
    for game_input in this_games_input:
        board_pos = 3*game_input['move'][1] + game_input['move'][0]
        this_game_board[board_pos] = game_input['player']
        
        if game_input['player'] == 2:
            current_board_key = past_boards_key(this_game_board)
            if current_board_key not in past_boards:
                past_boards[current_board_key] = [0] * 9
            
            if game_over_result[1]: # computer_win == True
                past_boards[current_board_key][board_pos] += 1
            else:
                past_boards[current_board_key][board_pos] -= 1
    
    return past_boards
            

def game_loop(past_boards):
    board = [None] * 9
    player = 1
    this_games_moves = []
    game_over_result = [False]
    while not game_over_result[0]:
        if player == 2:
            this_games_moves.append({'player':player, 'move': get_computer_move(board, past_boards)})
        else:
            player_move = get_player_move(player, board)
            if player_move != False and check_move_valid(player, player_move, board):  
                this_games_moves.append({'player':player, 'move':player_move})
                add_move_to_board(player, player_move, board)
            else:
                break
            
        player = switch_player(player)
        print_board(board)
        game_over_result = check_game_over(board)
        
    if game_over_result[0]:
        if game_over_result[1]:
            print("Computer wins!")
        else:
            print("Player 1 wins!")
        
    past_boards = score_past_boards(past_boards, this_games_moves, game_over_result)
        
    continue_input = input("Game over! Play again? [Y/n] ") 
    if continue_input == '' or continue_input == 'Y' or continue_input == 'y':
        return True
    return False
   

if __name__ == "__main__":
    try:
        with open('saved_moves.txt', 'rb') as handle:
            past_boards = pickle.loads(handle.read())
    except FileNotFoundError:
        past_boards = {}
        
    print("Enter your moves in the format 0,0 to place your mark on the upper left corner of the board")
    while game_loop(past_boards):
        pass
    
    with open('saved_moves.txt', 'wb') as handle:
        pickle.dump(past_boards, handle)
    
    
    

Enter your moves in the format 0,0 to place your mark on the upper left corner of the board
Player 1's turn: 1,1
   |   |   
---+---+---
   | X |   
---+---+---
   |   |   
Computer's turn...
   | O |   
---+---+---
   | X |   
---+---+---
   |   |   
Player 1's turn: 0,2
   | O |   
---+---+---
   | X |   
---+---+---
 X |   |   
Computer's turn...
 O | O |   
---+---+---
   | X |   
---+---+---
 X |   |   
Player 1's turn: 0,1
 O | O |   
---+---+---
 X | X |   
---+---+---
 X |   |   
Computer's turn...
 O | O |   
---+---+---
 X | X |   
---+---+---
 X |   | O 
Player 1's turn: 1,2
 O | O |   
---+---+---
 X | X |   
---+---+---
 X | X | O 
Computer's turn...
 O | O | O 
---+---+---
 X | X |   
---+---+---
 X | X | O 
Computer wins!
Game over! Play again? [Y/n] 
Player 1's turn: 1,1
   |   |   
---+---+---
   | X |   
---+---+---
   |   |   
Computer's turn...
   | O |   
---+---+---
   | X |   
---+---+---
   |   |   
Player 1's turn: 0,0
 X | O |   
---+---+---
   | X |   
---+--